## Installation
-  Run this only once

In [ ]:
!pip install gdown
!pip install tf_slim
!pip install numpy==1.22
!pip install LibRecommender


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip

## Restart 

In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [ ]:
SEED=42

In [ ]:
# !gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
# !unzip -o "data.zip"  -d  "/content"

# movie_10k_df = pd.read_csv("movies_min.csv")
# amazon_df =pd.read_csv('amazon_min.csv')
# book_df=pd.read_csv("book_min.csv")
# food_df = pd.read_csv('food_min.csv')
# ecommerce_df=pd.read_csv('ecom_min.csv')
# ciao_df=pd.read_csv('ciao_min.csv')

In [ ]:
TIMESTAMP=None

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct

In [ ]:
!unzip -o "data.zip"  -d  "/content"

## Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , timestamp** ]
- No issue if there is no column for **timestamp**

In [ ]:
filename="movies_min.csv" #enter your filename with file path

### Run below cell

In [ ]:
rating_df = pd.read_csv(filename)


In [ ]:
NUM_OF_COLS=rating_df.shape[1]


In [ ]:
import time 
if  'timestamp' in rating_df.columns:
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']
else:

  obj = time.gmtime(0)
  epoch = time.asctime(obj)
  curr_time = round(time.time()*1000)
  rating_df['timestamp']=curr_time
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [ ]:
u_users=list(rating_df.userID.unique())

In [ ]:
u_items=list(rating_df.itemID.unique())

##### Code Setup

In [ ]:
# Evaluation metrics
import libreco
import tensorflow as tf
import torch
from libreco.data import DatasetFeat, DataInfo
from libreco.data import  split_by_ratio
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats
from libreco.algorithms import (
    ALS
)
import time
import pandas as pd

metrics = [
    "precision",
    "recall",
    "map",
    "ndcg"
]

# Call Librecommender API
# datasettype ="pure" or "feat"
def run(rating_df,datasettype="pure", algo=["als"],hyperparam={"als":{ "embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}}, neg_samples=True):
    train_data, test_data, data_info=None,None,None
    tf.compat.v1.reset_default_graph()
    if(datasettype=="pure"):
      train_data, test_data, data_info = preprocessingPure(rating_df, neg_samples)
    else :
      train_data, test_data, data_info = preprocessingFeature(rating_df, neg_samples) 
    models = dict()
    evals = dict()
    if(neg_samples):
      train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
      test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    for i in algo:
        if i == 'als':
            
            als, eval, algo_data_info = alteringleastsquare(train_data, test_data, data_info,hyperparam["als"])
            models['als'] = als
            evals['als'] = eval
            print("Evaluation Results for ALS:")
            print(evals['als'])
        elif i == 'rnn4rec':
            rnn, eval, algo_data_info = rnn4rec(train_data, test_data, data_info,hyperparam["rnn4rec"])
            models['rnn4rec'] = rnn
            evals['rnn4rec'] = eval
            print("Evaluation Results for RNN4REC:")
            print(evals['rnn4rec'])
        elif i == 'bpr':
            bpr, eval, algo_data_info = bayesianpersonalizedranking(train_data, test_data, data_info,hyperparam["bpr"])
            models['bpr'] = bpr
            evals['bpr'] = eval
            print("Evaluation Results for BPR:")
            print(evals['bpr'])
        else:
            graphsage, eval, algo_data_info = graphSage(train_data, test_data, data_info,hyperparam["graphSage"])
            models['graphSage'] = graphsage
            evals['graphSage'] = eval
            print("Evaluation Results for GraphSage:")
            print(evals['graphSage'])
          
    return models, evals


# data preprocessing
def preprocessingPure(rating_df, neg_samples=True):
    data = rating_df.copy()
    data.columns = ['user', 'item', 'label', 'timestamp']
    col_user = ["user"]
    col_item = ["item"]
    col_dense = ["label"]
    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetPure.build_trainset(
        train,
        shuffle=False,
    )
    test_data = DatasetPure.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info
# data preprocessing
def preprocessingFeature(rating_df, neg_samples=True):
    data = rating_df.copy()
    data.columns = ['user', 'item', 'label', 'timestamp']
    col_user = ["user"]
    col_item = ["item"]
    col_dense = ["label"]
    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetFeat.build_trainset(
        train,
        shuffle=False,
    )
    test_data = DatasetFeat.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info


# Recommend items to user
def recommend(model, userid, k=10):
    return model.recommend_user(user=userid, n_rec=k)

# Find similar items
def similarItems(model, itemids, k=10):
    return model.search_knn_items(item=itemids, k=k)


# Model : Alternating Least Square (ALS)
def alteringleastsquare(train_data, test_data, data_info,hyperparam={"embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}):
    als = libreco.algorithms.ALS(
        "ranking",
        data_info,
        embed_size=hyperparam['embed_size'],
        n_epochs=hyperparam['n_epochs'],
        reg=hyperparam['reg'],
        alpha=hyperparam['alpha'],
        use_cg=hyperparam['use_cg'],
        n_threads=hyperparam['n_threads'],
        seed=hyperparam['seed'],
    )
    als.fit(
        train_data,
        verbose=2,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=als, data=test_data, metrics=metrics)
    return als, result, data_info

def graphSage(train_data, test_data, data_info,hyperparam={ "loss_type":"max_margin",
        "paradigm":"i2i",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "dropout_rate":0.0,
        "num_layers":1,
        "num_neighbors":10,
        "num_walks":10,
        "sample_walk_len":5,
        "margin":1.0,
        "sampler":"random",
        "start_node":"random",
        "focus_start":False,
        "seed":42}  ):
    graphsage = libreco.algorithms.GraphSage(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
        paradigm=hyperparam["paradigm"],
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        dropout_rate=hyperparam["dropout_rate"],
        num_layers=hyperparam["num_layers"],
        num_neighbors=hyperparam["num_neighbors"],
        num_walks=hyperparam["num_walks"],
        sample_walk_len=hyperparam["sample_walk_len"],
        margin=hyperparam["margin"],
        sampler=hyperparam["sampler"],
        start_node=hyperparam["start_node"],
        focus_start=hyperparam["focus_start"],
        seed=hyperparam["seed"],
    )
    graphsage.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=graphsage, data=test_data, metrics=metrics)
    return graphsage, result, data_info

def bayesianpersonalizedranking(train_data, test_data, data_info,hyperparam={ "loss_type":"bpr",
      
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True}  ):
    bpr = libreco.algorithms.BPR(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
     
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        use_tf=hyperparam["use_tf"],
    )
    bpr.fit(
        train_data,
        verbose=2,
        num_threads=4,
        eval_data=test_data,
        metrics=metrics,
        optimizer="adam",
    )
    result = evaluate(model=bpr, data=test_data, metrics=metrics)
    return bpr, result, data_info

def rnn4rec(train_data, test_data, data_info,hyperparam={ "rnn_type":"gru",
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}  ):
    rnn = libreco.algorithms.RNN4Rec(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
        rnn_type=hyperparam["rnn_type"],
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        recent_num=hyperparam["recent_num"],
        tf_sess_config=hyperparam["tf_sess_config"],
    )
    rnn.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=rnn, data=test_data, metrics=metrics)
    return rnn, result, data_info




    


Instructions for updating:
non-resource variables are not supported in the long term


## Run BPR

In [ ]:

models, evals = run(rating_df, algo=["bpr"], hyperparam={"bpr":{ "loss_type":"bpr",
        "paradigm":"i2i",
        "embed_size":32,
        "n_epochs":20,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":512,
        "num_neg":1,
        "use_tf":True} },neg_samples=True)


random neg item sampling elapsed: 0.090s
random neg item sampling elapsed: 0.022s
random neg item sampling elapsed: 0.170s
random neg item sampling elapsed: 0.047s
Training start time: 2023-02-07 20:12:58


pair_sampling train: 100%|██████████| 302/302 [00:00<00:00, 352.08it/s]


Epoch 1 elapsed: 0.900s


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 191.82it/s]


	 eval precision@10: 0.1802
	 eval recall@10: 0.0982
	 eval map@10: 0.3441
	 eval ndcg@10: 0.4563


...

Epoch 20 elapsed: 0.830s


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 164.32it/s]


	 eval precision@10: 0.2813
	 eval recall@10: 0.1792
	 eval map@10: 0.4929
	 eval ndcg@10: 0.6171


eval_listwise: 100%|██████████| 135/135 [00:00<00:00, 842.80it/s]


Evaluation Results for BPR:
{'precision': 0.2813361611876988, 'recall': 0.17917464781290038, 'map': 0.49289595070972897, 'ndcg': 0.6170735955532652}


#### Recommendations to users

In [ ]:
bpr=models['bpr']

In [ ]:
userids=u_users[0]
itemids=u_items[0]

In [ ]:
print('recommendation user '+str(userids))
results= recommend(bpr,1, 10)
print(results)

recommendation user 196
{1: array([204, 172, 168, 195,   7,  79, 423, 216, 405,  28])}


In [ ]:
print(" 10 most similar items for item : ", similarItems(bpr,itemids, k=10))

 10 most similar items for item :  [306, 242, 303, 301, 690, 302, 300, 323, 750, 319]
